In [1]:
!chmod a+x zhunt3-alan.c

In [2]:
!gcc zhunt3-alan.c -lm -o zhunt3

zhunt3-alan.c: In function ‘user_regret’:
zhunt3-alan.c:303:7: warning: implicit declaration of function ‘gets’; did you mean ‘fgets’? [-Wimplicit-function-declaration]
  303 |       gets( tempstr );
      |       ^~~~
      |       fgets
/usr/bin/ld: /tmp/cchmp4pJ.o: in function `user_regret':
zhunt3-alan.c:(.text+0xba7): warning: the `gets' function is dangerous and should not be used.


In [3]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.4 MB/s eta 0:00:00


In [4]:
import Bio

In [5]:
from Bio import SeqIO

In [6]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/galGal6/bigZips/galGal6.fa.gz

--2023-06-08 16:27:19--  https://hgdownload.soe.ucsc.edu/goldenPath/galGal6/bigZips/galGal6.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347959049 (332M) [application/x-gzip]
Saving to: ‘galGal6.fa.gz’

galGal6.fa.gz       100%[===================>] 331.84M   136MB/s    in 2.4s    

2023-06-08 16:27:22 (136 MB/s) - ‘galGal6.fa.gz’ saved [347959049/347959049]



In [7]:
!gunzip -d galGal6.fa.gz

In [8]:
input_file="galGal6.fa" 
fasta_sequence = SeqIO.parse(input_file,'fasta')
for record in SeqIO.parse(input_file, "fasta"): 
    if record.id == "chr20":
        with open(f"{record.id}.fa", "w") as file:
            SeqIO.write(record, file, "fasta")
        sequence=str(record.seq)
        break

In [19]:
!./zhunt3 12 8 12 chr20.fa chr20

dinucleotides 12
min/max 8 12
min/max 8 12
operating on chr20.fa
calculating zscore
opening chr20.fa
inputting sequence
opening chr20.fa.Z-SCORE
^C


In [11]:
import pandas as pd

1. Выбираем колонки с Z-score >= 400

In [24]:

z=pd.read_csv("chr20.fa.Z-SCORE", skiprows=1, names=["Start","End","1","2","3","Score","Seq","4"], delim_whitespace=True)

res = z.loc[z['Score'] >= 400]
display(res)

,Start,End,1,2,3,Score,Seq,4
24317,24318,24338,20,21.904,41.248,456.1458,gcttgcgttgcgcaggcgtg,SASASAASASASASASASAS
24319,24320,24338,18,21.711,42.703,571.7723,ttgcgttgcgcaggcgtg,SASAASASASASASASAS
24321,24322,24338,16,21.519,43.304,719.0200,gcgttgcgcaggcgtg,SAASASASASASASAS
24323,24324,24340,16,21.526,42.857,712.1870,gttgcgcaggcgtggc,ASASASASASASASSA
24325,24326,24342,16,21.527,42.829,711.6643,tgcgcaggcgtggcct,ASASASASASASSASA
...,...,...,...,...,...,...,...,...
13894027,13894028,13894046,18,21.006,43.114,1354.1140,tcgcgcatgcgtactgcc,SASASASASASASASASA
13894028,13894029,13894045,16,20.261,40.043,3612.6780,cgcgcatgcgtactgc,ASASASASASASSASA
13894029,13894030,13894048,18,21.069,42.486,1250.6400,gcgcatgcgtactgccgt,SASASASASASASASASA
13894831,13894832,13894850,18,21.979,51.240,418.3883,agcccacacgggcatggg,ASASASASASASASASAS


В полученной таблице 36086 строк - 36086 z-днк

2. Найдём количество квадруплексов на положительном стренде:

In [52]:
import re #для работы с регулярными выражениями 
input_file="chr20.fa" #путь к файлу
fasta_sequence = SeqIO.parse(input_file,'fasta') #импортируем fasta файл
for record in SeqIO.parse(input_file, "fasta"):  #для каждой записи из файла 
 name, sequence = record.id, str(record.seq) 
pattern="(?:G{3,}[ATGC]{1,7}){3,}G{3,}" #шаблон стандартного квадруплекса на + стренде
PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern,sequence,re.IGNORECASE)] #re.IGNORECASE отвечает за игнорирования отличий между заглавными и строчными буквами
len(PQS) #выводим число найденных квадруплексов


1554

На отрицательном:

In [53]:
pattern_C="(?:C{3,}[ATGC]{1,7}){3,}C{3,}"
PQS_minus=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_C,sequence,re.IGNORECASE)] #re.IGNORECASE отвечает за игнорирования отличий между заглавными и строчными буквами
len(PQS_minus) #выводим число найденных квадруплексов

1439

Всего квадруплексов:

In [58]:
G4 = PQS + PQS_minus
len(G4)

2993

3.

In [37]:
table = pd.read_csv("Genes  NCBI Gallus gallus Annotation Release 104  2018-05-17.CSV", skiprows=2, names=["Accession","Start","Stop","Gene symbol","Strand","NCBI Gene ID","Name"], sep = ",")
display(table)


,Accession,Start,Stop,Gene symbol,Strand,NCBI Gene ID,Name
0,NC_006107.5,71558,75275,IGSF1L3,plus,419114,immunoglobulin superfamily member 1-like 3
1,NC_006107.5,76641,93864,L3MBTL1,minus,419115,NaN
2,NC_006107.5,119215,123854,SRSF6,minus,419116,serine/arginine-rich splicing factor 6
3,NC_006107.5,187678,257245,EPB41L1,plus,419117,NaN
4,NC_006107.5,256427,296024,AAR2,plus,419118,AAR2 splicing factor homolog
...,...,...,...,...,...,...,...
451,NC_006107.5,13795911,13801004,RNF114,minus,414881,ring finger protein 114
452,NC_006107.5,13801269,13807900,SPATA2,plus,419355,spermatogenesis associated 2
453,NC_006107.5,13817409,13817494,MIR1648,plus,100315762,microRNA 1648
454,NC_006107.5,13824723,13847411,SLC9A8,minus,419356,solute carrier family 9 member A8


In [30]:
gene_count = 0
intersection_count = 0
intergenic_count = 0
for index2, row2 in res.iterrows():
  gene_hit = 0
  intersection_hit = 0
  for index1, row1 in table.iterrows():
    if row2["Start"] >=  row1["Start"] and row2["End"] <= row1["Stop"]:
      gene_hit = 1
      break
    elif (row2["Start"] >= row1["Start"] and row2["Start"] <= row1["Stop"]) or (row2["End"] >= row1["Start"] and row2["End"] <= row1["Stop"]):
      intersection_hit = 1
  if gene_hit:
      gene_count+=1
  elif intersection_hit:
    intersection_count+=1
  else:
    intergenic_count+=1
print(gene_count)
print(intersection_count)
print(intergenic_count)

21748
373
13965


In [ ]:
print(gene_count/len(res)*100)
print(intergenic_count/len(res)*100)
print(intersection_count/len(res)*100)

60.26713961092944
38.6992185335033
1.033641855567256


60,3% z-днк попадают в гены,
38,7% z-днк попадают в межгеное пространство,
1% z-днк попадает в пересечение

In [60]:
gene_count = 0
intersection_count = 0
intergenic_count = 0
for row2 in G4:
  gene_hit = 0
  intersection_hit = 0
  for index1, row1 in table.iterrows():
    if row2[0] >=  row1["Start"] and row2[1] <= row1["Stop"]:
      gene_hit = 1
      break
    elif (row2[0] >= row1["Start"] and row2[0] <= row1["Stop"]) or (row2[1] >= row1["Start"] and row2[1] <= row1["Stop"]):
      intersection_hit = 1
  if gene_hit:
      gene_count+=1
  elif intersection_hit:
    intersection_count+=1
  else:
    intergenic_count+=1
print(gene_count)
print(intersection_count)
print(intergenic_count)
  

1886
27
1080


In [62]:
print(gene_count/len(G4)*100)
print(intergenic_count/len(G4)*100)
print(intersection_count/len(G4)*100)

63.013698630136986
36.08419645840294
0.9021049114600734


63% квадруплексов попадают в гены, 36% квадруплексов попадают в межгеное пространство, 0.9% квадруплексов попадает в пересечение

In [46]:
promoter_names = []
promoter_starts = []
promoter_ends = []
for index, row in table.iterrows():
  if row["Strand"] == "plus":
    promoter_names.append(row["Gene symbol"])
    promoter_starts.append(row["Start"]-1000)
    promoter_ends.append(row["Start"])
  if row["Strand"] == "minus":
    promoter_names.append(row["Gene symbol"])
    promoter_starts.append(row["Start"])
    promoter_ends.append(row["Start"]+1000)

In [47]:
print(promoter_starts)
print(promoter_ends)

[70558, 76641, 119215, 186678, 255427, 315103, 438322, 477158, 483271, 485849, 493851, 496720, 520841, 560718, 565878, 600455, 621418, 630014, 638622, 654886, 658642, 680594, 765131, 828509, 839126, 895380, 903547, 977605, 983084, 998740, 1026890, 1030053, 1061506, 1062494, 1078706, 1078716, 1122962, 1159065, 1180450, 1188538, 1215851, 1254929, 1274269, 1333629, 1358767, 1366140, 1419424, 1422870, 1436200, 1451735, 1458845, 1529232, 1547822, 1567238, 1667700, 1689069, 1709366, 1938216, 2000275, 2006395, 2010364, 2073710, 2120742, 2146536, 2160706, 2165643, 2176137, 2181993, 2208409, 2272654, 2299286, 2517821, 2539619, 2543313, 2556331, 2591000, 2603245, 2623446, 2633595, 2669027, 2837187, 3167217, 3291913, 3390518, 3426190, 3445995, 3611963, 3626252, 3630044, 3642069, 3658587, 3694821, 3733585, 3737686, 3765932, 3813506, 3821376, 3825903, 4137830, 4311875, 4361046, 4381090, 4444981, 4554027, 4601622, 4690672, 4748891, 4756442, 4784500, 4834314, 4895639, 4931004, 4940558, 4971781, 50187

In [22]:
!unzip chr20.fa.zip

Archive:  chr20.fa.zip
  inflating: chr20.fa.Z-SCORE        


4. Cписок генов, в промоторы которых попадает Z-ДНК:

In [50]:
for i in range(len(promoter_names)):
  for index, row in res.iterrows():
    if row["Start"] >= promoter_starts[i] and  row["End"] <= promoter_ends[i]:
      print(promoter_names[i])
      break

  

DLGAP4
LOC112530104
TGIF2
LOC112530107
MYL9
C20H20ORF24
NCOA6
LOC112530130
TP53INP2
LOC107054815
RBL1
RALGAPB
SLA2
PHF20
TRNAE-UUC
NFS1
CPNE1
RBM12
FER1L4
DDEFL1
GDF5
UQCC1
EIF6
MAP1LC3A
MIR1453
AHCY
ASIP
EIF2S2
RALY
ZNF341
PXMP4
E2F1
NECAB3
CBFA2T2
SNTA1
EDEM2
TRPC4AP
MYH7B
GSS
SOGA1
PTPRT
LOC101749104
LOC101749307
FAM83D
LOC101750504
TOP1
PLCG1
MIR1614
EMILIN3
RPN2
SRC
PIGT
DBNDD2
TP53TG5
SDC4
TOMM34
R3HDML
LOC107057318
OSER1
TP53RK
SLC2A10
NCOA3
CSE1L
SNORD12C
LOC112529916
LOC107054873
ZNFX1
PHACTR3
FAM217B
CDH4
LOC107054870
LSM14B
SS18L1
OSBPL2
MIR1-1
LOC112530111
NTSR1
MRGBP
OGFR
COL9A3
MIR6567
MIR1746
BIRC7
DEPTOR
CHRNA4
KCNQ2
EEF1A2
PPDPF
LOC107054863
LOC107054862
LOC419250
RGS19
SOX18
LOC100858256
SAMD10
LOC107054857
UCKL1
LOC107054856
ZBTB46
SLC2A4RG
LIME1
ZGPAT
ARFRP1
TNFRSF6B
STMN3
PDYN
SIRPA
FKBP1C
SNPH
LOC112530133
TMEM74B
RSPO4
ANGPT4
LOC112530115
FAM110A
SLC52A3
SRXN1
TCF15
CSNK2A1
TBC1D20
SOX12
LOC101748049
C20H20orf96
REM1
LOC101748448
HM13
ID1
BPIL
BCL2L1
TPX2
IRF9
LO

5.Cписок генов, в промоторы которых попадает квадруплексы.

In [64]:
for i in range(len(promoter_names)):
  for item in G4:
    if item[0] >= promoter_starts[i] and item[1] <= promoter_ends[i]:
      print(promoter_names[i])
      break


SRSF6
EPB41L1
DLGAP4
TGIF2
LOC112530107
MYL9
NCOA6
LOC107054815
RALGAPB
LOC107057310
PHF20
NFS1
CPNE1
RBM12
DDEFL1
AHCY
EIF2S2
RALY
LOC107054897
PXMP4
E2F1
SNTA1
EDEM2
MYH7B
GSS
PTPRT
LOC101749307
ARHGAP40
PPP1R16B
MAFB
SDC4
RBPJL
MATN4
PI3
KCNS1
TOMM34
RIMS4
ADA
SERINC3
HNF4A
R3HDML
LOC112530126
LOC107054877
TP53RK
SLC2A10
STAU1
SNORD12C
FAM217B
CDH4
LSM14B
SS18L1
OSBPL2
ADRM1
RBBP8NL
MIR1-1
SLCO4A1
LOC771200
LOC112530118
MIR124-3
YTHDF1
BIRC7
ARFGAP1
DEPTOR
LOC107054822
NPBWR2
RGS19
SOX18
LOC100858256
C20H20orf204
PRPF6
SAMD10
ZNF512B
UCKL1
LOC107054855
LOC107054856
TPD52L2
ZBTB46
SLC2A4RG
LIME1
STMN3
GMEB2
HELZ2
STK35
TPPNR1L
SIRPA
NSFL1C
FKBP1C
SNPH
TMEM74B
LOC107054820
RSPO4
ANGPT4
SLC52A3
SCRT2
SRXN1
TCF15
TBC1D20
NRSN2
SOX12
LOC101748049
C20H20orf96
REM1
HM13
ID1
BPIL
TPX2
FOXS1
IRF9
LOC771689
TTLL9
XKR7
CCM2L
HCK
POFUT1
KIF3B
C20H20ORF112
LOC107054851
COMMD7
DNMT3B
MAPRE1
LBP
TGM2
TTI1
VSTM2L
WFDC2L
WFDC2
WFDC8
LOC107054850
UBE2C
LOC107054848
SNX21
CTSA
PCIF1
SLC12A5
CD40
SNRPL